This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json 
import statistics

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?rows=1&key='+API_KEY
r = requests.get(url)
json_afx1= r.json() #returns dict 

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(json.dumps(json_afx1, sort_keys=True, indent=4)) # .dumps() makes things more readable, esp nesting structure

{
    "dataset": {
        "collapse": null,
        "column_index": null,
        "column_names": [
            "Date",
            "Open",
            "High",
            "Low",
            "Close",
            "Change",
            "Traded Volume",
            "Turnover",
            "Last Price of the Day",
            "Daily Traded Units",
            "Daily Turnover"
        ],
        "data": [
            [
                "2020-12-01",
                112.2,
                112.2,
                111.5,
                112.0,
                null,
                51.0,
                5703.0,
                null,
                null,
                null
            ]
        ],
        "database_code": "FSE",
        "database_id": 6129,
        "dataset_code": "AFX_X",
        "description": "Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704",
        "end_date": "2020-12-01",
        

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

## Code Task 1: *Collect data from FSE for AFX_X for 2017
### AND
## Code Task 2: *Convert the returns JSON object into a Python Dictionary

In [5]:
# Answer 1
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&key='+API_KEY
r = requests.get(url)

# Answer 2
afx2017= r.json()

## Code Task 3: *Calculate what the highest and lowest opening prices were for AFX_X in this period (2017)*

In [6]:
data_list = afx2017['dataset']['data'] #nested lists of each day's data
open_date = {}
for i in range(len(data_list)):
    if data_list[i][1] != None: # if statement necessary bc no >/< on NoneType
        open_date[data_list[i][0]] = (data_list[i][1]) # key:value pr Date:Open

highest_open = max(open_date.values())
lowest_open = min(open_date.values())

In [7]:
# Answer 3
print('The highest and lowest opening prices for AFX_X are $' + str(highest_open), 'and $'+ str(lowest_open) + ', respectively.' )

The highest and lowest opening prices for AFX_X are $53.11 and $34.0, respectively.


## Code Task 4: *What was the largest change in any one day (based on High and Low)?*

In [8]:
date_HL = {} # Date:[High, Low]
for i in range(len(data_list)):
    if data_list[i][2] != None and data_list[i][3] != None:
        date_HL[data_list[i][0]] = [data_list[i][2], data_list[i][3]]

changes = {} # Date:high-low
for date, HL in date_HL.items(): #HL is list of High, Low
    change = HL[0] - HL[1] #High - Low
    changes[date] = change

In [9]:
# Answer 4
print('The largest change in any one day based on the difference between highest price and lowest price was $' + 
      str(round(max(changes.values()), 2)) + '.')

The largest change in any one day based on the difference between highest price and lowest price was $2.81.


## Code Task 5: *What was the largest change between any two days (based on Close Price)?*

In [10]:
close_date = {} # Date:Close
for i in range(len(data_list)):
    if data_list[i][1] != None:
        close_date[data_list[i][0]] = data_list[i][4]

In [11]:
just_close = list(close_date.values()) # list of close $
close_deltas = [] # list of deltas between days
for x, y in zip(just_close[0::], just_close[1::]): # makes tuple {(day0, day1), (day1, day2), ...}
    close_deltas.append(y-x)  # (dayn) - (dayn-1)

In [12]:
# Answer 5
print("The largest changed between any two days' closing price was $" +str(round(max(close_deltas),2))+'.')

The largest changed between any two days' closing price was $2.56.


## Code Task 6: *What was the average daily trading volume during this year?*

In [13]:
volume_list = [] # list of each day's trading volume
for i in range(len(data_list)):
    if data_list[i][6] != None:
        volume_list.append(data_list[i][6])

In [14]:
# Answer 6 
print("The average daily trading volume during 2017 is " +str(statistics.mean(volume_list))+' shares.')

The average daily trading volume during 2017 is 89124.33725490196 shares.


## (Optional) Code Task 7: *What was the median trading volume during this year.*

In [15]:
median_volume = str(statistics.median(volume_list))

In [16]:
# Answer 7
print("The median trading volume during 2017 was",median_volume,"shares.")

The median trading volume during 2017 was 76286.0 shares.
